# kNN Hash Example

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter

## Iris dataset

In [9]:
df = load_iris()
df.data.shape
datf = pd.DataFrame(df.data, columns=df.feature_names)
print(df.data.shape)

(150, 4)


In [15]:
datf

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [11]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

* https://docs.python.org/2/library/functools.html Here you can read about "partial"
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html About mapping to [0,1]

In [12]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index
    
    def predict(self,u):
        
        for j in range(self.L):
            inds = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            
#             e_distance = np.sqrt(np.sum((X-point)**2,axis = 1))
            
            #Here you must put your code, extend the method with distance function and calculation with unknown sample "u"
            #Develop the rest part of kNN predict method that was discussed at the lecture

In [13]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target


In [14]:
knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict(test1x)
print("-------------")
print(test2y)
knnhash.predict(test2x)
print("-------------")
print(test3y)
knnhash.predict(test3x)

0
Counter({0: 31, 1: 6})
Counter({0: 18, 1: 1})
Counter({0: 6, 1: 2})
Counter({0: 20})
-------------
1
Counter({1: 23, 2: 6, 0: 1})
Counter({1: 6, 2: 3})
Counter({1: 6, 2: 6})
Counter({1: 13, 2: 13})
-------------
2
Counter({2: 8, 1: 3})
Counter({2: 2, 1: 1})
Counter({1: 8, 2: 3})
Counter({1: 13, 2: 13})


* Each string above corresponds to the particular hash table. And index in counter maps to the class. For example Counter({0: 13, 1: 1}) means that there are 13 samples close to "u" with "0" class labels and 1 sample with "1" class label.